In [10]:
#TODOS LOS IMPORTS
# data manipulation and plotting
import pandas as pd
import numpy as np

# for saving the pipeline
import joblib

# from Scikit-learn
from sklearn.pipeline import Pipeline

# from feature-engine
from feature_engine.imputation import (
    MeanMedianImputer,
    AddMissingIndicator
)

from feature_engine.selection import DropFeatures

#to separate training and test
from sklearn.model_selection import train_test_split
#the model
from sklearn.naive_bayes import GaussianNB



In [11]:

# CARGAR DATA
file_name = "/course/datasets/heights_weights_con_cosas.csv"
df = pd.read_csv(file_name)
df.head()

,Height,Weight,Male,Age,Country
0,73.847017,241.893563,1.0,25.0,Spain
1,68.781904,162.310473,1.0,25.0,Spain
2,74.110105,212.740856,1.0,25.0,Spain
3,71.730978,220.042470,1.0,25.0,Spain
4,69.881796,206.349801,1.0,25.0,Spain


In [12]:
X = df[["Height", "Weight","Age","Country"]]
Y = df["Male"]

In [13]:
# Separamos entre train y test

X_train, X_test, Y_train, Y_test = train_test_split(
    X, #features
    Y, #labels
    test_size=0.3, # % de test
    random_state=0 #seed 
)

X_train = pd.DataFrame(X_train, columns = ['Height','Weight','Age','Country'])
X_test = pd.DataFrame(X_test, columns = ['Height','Weight','Age','Country'])
Y_train = pd.DataFrame(Y_train, columns = ['Male'])
Y_test = pd.DataFrame(Y_test, columns = ['Male'])


In [14]:
#Configuración

# Variables númericas con NaNs en train set
NUMERICAL_VARS_WITH_NA = ['Height', 'Weight']


# Drop features (hay que hacer analisis de correlación)
DROP_FEATURES = ['Age','Country']

# Variables seleccionadas
FEATURES = [
    'Height',
    'Weight'
]

In [15]:
# Creamos el pipe
genero_pipe = Pipeline([

    # ===== IMPUTATION =====
    ('drop_features', DropFeatures(features_to_drop=DROP_FEATURES)),
    #  missing indicator
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),

    # imputamos variables numericas con la mean
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA
    )),
    ('GaussianNB', GaussianNB())
   
   ])

In [16]:
# Entrenamos el pipe
genero_pipe.fit(X_train, Y_train.values.ravel())

Pipeline(steps=[('drop_features',
                 DropFeatures(features_to_drop=['Age', 'Country'])),
                ('missing_indicator',
                 AddMissingIndicator(variables=['Height', 'Weight'])),
                ('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Height', 'Weight'])),
                ('GaussianNB', GaussianNB())])

In [17]:
ejemplo = pd.DataFrame([[64.3103463968169,142.821243757182,25,"Spain"]], columns = ['Height','Weight','Age','Country'])
pred = genero_pipe.predict(ejemplo)
print(pred[0])

0.0


In [18]:
joblib.dump(genero_pipe, open('/course/datasets/cfk.pkl', 'wb'))

In [21]:
file_name = "/course/datasets/test.csv"
testing = pd.read_csv(file_name)
df.head()

,Height,Weight,Male,Age,Country
0,73.847017,241.893563,1.0,25.0,Spain
1,68.781904,162.310473,1.0,25.0,Spain
2,74.110105,212.740856,1.0,25.0,Spain
3,71.730978,220.042470,1.0,25.0,Spain
4,69.881796,206.349801,1.0,25.0,Spain


In [22]:
pred = genero_pipe.predict(testing)

In [24]:
type(pred)

numpy.ndarray

In [23]:
print(pred)

[1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
